In [121]:
import pymysql.cursors
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy import text
import getpass

In [122]:
#prompt user to enter MySQL root password
sql_pass = getpass.getpass()
#create connection string and engine to connect to MySQL database
connection_string = 'mysql+pymysql://root:' + sql_pass + '@localhost:3306/civictech'
engine = create_engine(connection_string)

In [123]:
import pandas as pd

In [124]:
people=pd.read_csv('profiles/profile_data_reprocessed.csv', sep=';') # possible index drop
people.head()

,profile_id,lastName,firstName,geoCountryName,geoLocationName,birthDate,summary,industryName,headline,experience,...,ed3_grade,ed3_timePeriod,ed3_degreeName,ed3_schoolName,ed3_fieldOfStudy,ed3_school,ed3_description,ed3_activities,ed3_courses,ed3_honors
0,ACoAAAA615EBaNquQR5gOz_oFr9emeCr0ZNw67M,Brachet,Antoine,France,Greater Paris Metropolitan Region,NaN,Antoine croit à l’intelligence de tous et à la...,IT Services and IT Consulting,"Directeur associé de bluenove, initiateur du m...","[{'locationName': 'Paris Area, France', 'entit...",...,Mention très bien,"{'endDate': {'year': 1996}, 'startDate': {'yea...",Baccalauréat,Lycée Kernanec,Economics,NaN,NaN,NaN,NaN,NaN
1,ACoAAAfK9YwBjZr16cDaVuxZICOg0QUnbPiUoXE,Demri,Bobby,France,"Paris, Île-de-France",NaN,French Entrepreneur - Founder and Managing Par...,Venture Capital and Private Equity Principals,Founder & Managing Partner at ROCH Ventures,[{'entityUrn': 'urn:li:fs_position:(ACoAAAfK9Y...,...,NaN,"{'endDate': {'year': 2005}, 'startDate': {'yea...",Baccalauréat,Lycée Janson-de-Sailly,Sciences Economiques et Sociales Option Anglais,"{'objectUrn': 'urn:li:school:245045', 'entityU...",NaN,NaN,NaN,NaN
2,ACoAAAUn_5ABqO0mSShQxo4gFyTaCDoaYUk5Fm8,Durieux,Sarah,France,"Paris, Île-de-France",NaN,Inspired by the power we can build to change t...,Civic and Social Organizations,Co-director Multitudes Foundation - Activist a...,[{'entityUrn': 'urn:li:fs_position:(ACoAAAUn_5...,...,NaN,"{'endDate': {'month': 1, 'year': 2021}, 'start...",Public narrative,Leading change network - Marshall Ganz,Leadership organisationnel,NaN,Public narrative is a leadership-development p...,NaN,NaN,NaN
3,ACoAAARi9fcBfH1GjbFHnWc1QqHftfh3LQjRUEg,Jaillot,Bastien,France,Greater Paris Metropolitan Region,NaN,"IT Consultant, Technical Expert, Lead develope...",IT Services and IT Consulting,Web expert chez Jolicode,[{'entityUrn': 'urn:li:fs_position:(ACoAAARi9f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACoAAApOhv4B3_GF-OGg8-ipxxkMf6AOggWigl8,Vanneroy,Coline,France,Greater Paris Metropolitan Region,NaN,NaN,IT Services and IT Consulting,Directrice des opérations chez Cap Collectif,[{'entityUrn': 'urn:li:fs_position:(ACoAAApOhv...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
people.to_sql('people', engine, 'civictech', if_exists='replace', index=False)

47

In [126]:
companies_finance=pd.read_csv('scraping_organizations/companies_finance_data.csv', sep=';') # possible index drop
print(companies_finance.columns)

Index(['Unnamed: 0', 'Chiffre d'affaires (€)', 'Marge brute (€)',
       'EBITDA - EBE (€)', 'Résultat d'exploitation (€)', 'Résultat net (€)',
       'Croissance', 'Taux de croissance du CA (%)', 'Taux de marge brute (%)',
       'Taux de marge d'EBITDA (%)', 'Taux de marge opérationnelle (%)',
       'Gestion BFR', 'BFR (€)', 'BFR exploitation (€)',
       'BFR hors exploitation (€)', 'BFR (j de CA)',
       'BFR exploitation (j de CA)', 'BFR hors exploitation (j de CA)',
       'Délai de paiement clients (j)', 'Délai de paiement fournisseurs (j)',
       'Ratio des stocks / CA (j)', 'Autonomie financière',
       'Capacité d'autofinancement (€)', 'Capacité d'autofinancement / CA (%)',
       'Fonds de roulement net global (€)', 'Couverture du BFR',
       'Trésorerie (€)', 'Dettes financières (€)', 'Capacité de remboursement',
       'Ratio d'endettement (Gearing)', 'Autonomie financière (%)',
       'Taux de levier (DFN/EBITDA)', 'Solvabilité',
       'Etat des dettes à 1 an au plu

In [127]:
import inflection
import re
new_col_names=[]
for i in list(companies_finance.columns):
    i=i.lower().replace(' de', '_').replace(' du', '_').replace("'", '').replace(' ', '_').replace('é', 'e').replace('(€)','e').replace('(%)','pc').replace('/','sur').replace('à', 'a').replace('è', 'e').replace('(', '').replace(')', '').replace('ô', 'o').replace('__', '_').replace('s_t', 'surdet')
    new_col_names.append(inflection.underscore(i))
print(new_col_names)

['unnamed:_0', 'chiffre_daffaires_e', 'marge_brute_e', 'ebitda___ebe_e', 'resultat_dexploitation_e', 'resultat_net_e', 'croissance', 'taux_croissance_ca_pc', 'taux_marge_brute_pc', 'taux_marge_debitda_pc', 'taux_marge_operationnelle_pc', 'gestion_bfr', 'bfr_e', 'bfr_exploitation_e', 'bfr_hors_exploitation_e', 'bfr_j_ca', 'bfr_exploitation_j_ca', 'bfr_hors_exploitation_j_ca', 'delai_paiement_clients_j', 'delai_paiement_fournisseurs_j', 'ratio_s_stocks_sur_ca_j', 'autonomie_financiere', 'capacite_dautofinancement_e', 'capacite_dautofinancement_sur_ca_pc', 'fonds_roulement_net_global_e', 'couverture_bfr', 'tresorerie_e', 'dettes_financieres_e', 'capacite_remboursement', 'ratio_dendettement_gearing', 'autonomie_financiere_pc', 'taux_levier_dfnsurebitda', 'solvabilite', 'etat_surdettes_a_1_an_au_plus_e', 'liquidite_generale', 'couverture_surdettes', 'fonds_propres_e', 'rentabilite', 'marge_nette_pc', 'rentabilite_sur_fonds_propres_pc', 'rentabilite_economique_pc', 'valeur_ajoutee_e', 'valeu

In [128]:
companies_finance.columns = new_col_names

In [129]:
companies_finance.columns

Index(['unnamed:_0', 'chiffre_daffaires_e', 'marge_brute_e', 'ebitda___ebe_e',
       'resultat_dexploitation_e', 'resultat_net_e', 'croissance',
       'taux_croissance_ca_pc', 'taux_marge_brute_pc', 'taux_marge_debitda_pc',
       'taux_marge_operationnelle_pc', 'gestion_bfr', 'bfr_e',
       'bfr_exploitation_e', 'bfr_hors_exploitation_e', 'bfr_j_ca',
       'bfr_exploitation_j_ca', 'bfr_hors_exploitation_j_ca',
       'delai_paiement_clients_j', 'delai_paiement_fournisseurs_j',
       'ratio_s_stocks_sur_ca_j', 'autonomie_financiere',
       'capacite_dautofinancement_e', 'capacite_dautofinancement_sur_ca_pc',
       'fonds_roulement_net_global_e', 'couverture_bfr', 'tresorerie_e',
       'dettes_financieres_e', 'capacite_remboursement',
       'ratio_dendettement_gearing', 'autonomie_financiere_pc',
       'taux_levier_dfnsurebitda', 'solvabilite',
       'etat_surdettes_a_1_an_au_plus_e', 'liquidite_generale',
       'couverture_surdettes', 'fonds_propres_e', 'rentabilite',
     

In [130]:
str1='30,7K'
import re
re.split('[,K]', str1)
str2= int(re.split('[,K]', str1)[0]+re.split('[,K]', str1)[1]+'0'*(3-len(re.split('[,K]', str1)[1])))
str2

30700

In [131]:
str3='527K'
str4= int(str3.replace('K', '000'))
str4

527000

In [136]:
def fix_thousands(x):
    x=str(x)
    if 'K' in x:
        if ',' in x:
            return int(re.split('[,K]', x)[0]+re.split('[,K]', x)[1]+'0'*(3-len(re.split('[,K]', x)[1])))
        else:
            return int(x.replace('K', '000'))
    if 'M' in x:
        if ',' in x:
            return int(re.split('[,M]', x)[0]+re.split('[,M]', x)[1]+'0'*(6-len(re.split('[,M]', x)[1])))
        else:
            return int(x.replace('M', '000'))
    else:
        return x

In [137]:
companies_finance['chiffre_daffaires_e'].apply(fix_thousands)

0         nan
1       30700
2       46300
3       16700
4        1740
       ...   
69        134
70    1330000
71     824000
72     433000
73     259000
Name: chiffre_daffaires_e, Length: 74, dtype: object

In [119]:
a= list(companies_finance.columns).pop(0)

In [138]:
for i in list(companies_finance.columns)[1:]: 
    companies_finance[i]= companies_finance[i].apply(fix_thousands)

In [139]:
companies_finance

,unnamed:_0,chiffre_daffaires_e,marge_brute_e,ebitda___ebe_e,resultat_dexploitation_e,resultat_net_e,croissance,taux_croissance_ca_pc,taux_marge_brute_pc,taux_marge_debitda_pc,...,marge_nette_pc,rentabilite_sur_fonds_propres_pc,rentabilite_economique_pc,valeur_ajoutee_e,valeur_ajoutee_sur_ca_pc,structure_dactivite,salaires_et_charges_sociales_e,salaires_sur_ca_pc,impots_et_taxes_e,chiffre_daffaires_a_lexport_e
0,2017_citility,nan,nan,nan,nan,-562000,2017.0,nan,nan,nan,...,nan,nan,nan,nan,nan,2017.0,nan,nan,nan,nan
1,2016_citility,30700,527000,-295000,-296000,-238000,2016.0,4,1720,-961,...,-775,-623,-173,351000,1140,2016.0,636000,2070,9160,nan
2,2019_voxcracy,46300,46300,-31500,-38500,-39000,2019.0,177,100,-682,...,-842,-76,-63,2170,47,2019.0,33600,725,137,nan
3,2018_voxcracy,16700,157000,-123000,-130000,-112000,2018.0,861,940,-734,...,-668,-202,-169,-21000,-126,2018.0,101000,602,1120,5000
4,2017_voxcracy,1740,75900,-24400,-30300,-23400,2017.0,nan,4360,-1400,...,-1350,-41,-39,-5040,-290,2017.0,19100,1100,329,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2018_Stig,134,135,-9860,-11500,-2040,2018.0,nan,101,-7360,...,-1520,-915,-892,-9860,-7360,2018.0,nan,0,nan,nan
70,2021_opensourcepolitics,1330000,1340000,277000,284000,235000,2021.0,614,101,208,...,177,494,314,1030000,771,2021.0,793000,596,7030,160000
71,2020_opensourcepolitics,824000,686000,205000,188000,154000,2020.0,905,832,249,...,186,569,305,658000,798,2020.0,462000,561,4050,123000
72,2019_opensourcepolitics,433000,433000,65900,64200,60200,2019.0,67,100,152,...,139,517,222,326000,754,2019.0,279000,644,1620,nan


In [140]:
companies_finance.dtypes

unnamed:_0                             object
chiffre_daffaires_e                    object
marge_brute_e                          object
ebitda___ebe_e                         object
resultat_dexploitation_e               object
resultat_net_e                         object
croissance                             object
taux_croissance_ca_pc                  object
taux_marge_brute_pc                    object
taux_marge_debitda_pc                  object
taux_marge_operationnelle_pc           object
gestion_bfr                            object
bfr_e                                  object
bfr_exploitation_e                     object
bfr_hors_exploitation_e                object
bfr_j_ca                               object
bfr_exploitation_j_ca                  object
bfr_hors_exploitation_j_ca             object
delai_paiement_clients_j               object
delai_paiement_fournisseurs_j          object
ratio_s_stocks_sur_ca_j                object
autonomie_financiere              

In [141]:
companies_finance.to_sql('companies_finance', engine, 'civictech', if_exists='replace', index=False)

74

In [66]:
companies_info=pd.read_csv('scraping_organizations/companies_info_data.csv', sep=';') # possible index drop
print(companies_info.columns)

Index(['Unnamed: 0', 'Adresse :', 'Activité :', 'Effectif :', 'Création :',
       'Dirigeants :', 'SIREN :', 'SIRET (siège) :', 'Forme juridique :',
       'Numéro de TVA:', 'Inscription au RCS :', 'Numéro RCS :',
       'Capital social :', 'Activité principale déclarée :',
       'Code NAF ou APE :', 'Domaine d’activité :', 'Dirigeant :',
       'Capital variable (minimum) :', 'Convention collective  :',
       'Inscription au RNA :', 'Identifiant association :',
       'Objet de l'association :', 'Statut INSEE :'],
      dtype='object')


In [67]:
new_cols=[]
for i in list(companies_info.columns):
    i=i.lower().replace(' :', '').replace('é', 'e').replace("(", '').replace(')', '').replace(' ', '_').replace("'",'').replace("’",'')
    new_cols.append(i)
print(new_cols)

['unnamed:_0', 'adresse', 'activite', 'effectif', 'creation', 'dirigeants', 'siren', 'siret_siège', 'forme_juridique', 'numero_de_tva:', 'inscription_au_rcs', 'numero_rcs', 'capital_social', 'activite_principale_declaree', 'code_naf_ou_ape', 'domaine_dactivite', 'dirigeant', 'capital_variable_minimum', 'convention_collective_', 'inscription_au_rna', 'identifiant_association', 'objet_de_lassociation', 'statut_insee']


In [68]:
companies_info.columns=new_cols
companies_info.head()

,unnamed:_0,adresse,activite,effectif,creation,dirigeants,siren,siret_siège,forme_juridique,numero_de_tva:,...,activite_principale_declaree,code_naf_ou_ape,domaine_dactivite,dirigeant,capital_variable_minimum,convention_collective_,inscription_au_rna,identifiant_association,objet_de_lassociation,statut_insee
0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),05/05/2014,"André MAY, ODICEO, Sabine SCHNECK",802 503 276,802 503 276 00023,"SAS, société par actions simplifiée",FR07802503276,...,Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),01/09/2015,NaN,813 463 601,813 463 601 00027,"SAS, société par actions simplifiée",FR19813463601,...,Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",Philippe GERARD,NaN,NaN,NaN,NaN,NaN,NaN
2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,01/09/2014,"Olivier ROCCA, Pascal RUSCICA",803 548 031,803 548 031 00019,"SAS, société par actions simplifiée",FR08803548031,...,"La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",NaN,"100,00 €",NaN,NaN,NaN,NaN,NaN
3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),16/01/2015,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO",809 375 744,809 375 744 00012,"SAS, société par actions simplifiée",FR61809375744,...,Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),23/01/2008,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole...",502 204 944,502 204 944 00057,"SAS, société par actions simplifiée",FR77502204944,...,L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
companies_info.to_sql('companies_info', engine, 'civictech', if_exists='replace', index=False)

76